In [ ]:
### Chapter 11 Data Strucutures and Algorithms 
#### Donovan Manogue

In [ ]:
 The three most fundamental methods of a map M (see Section 10.1.1) are:

| Operation      | Description                                                                                  | Implemented With     |
|----------------|----------------------------------------------------------------------------------------------|-----------------------|
| `M[k]`         | Return the value `v` associated with key `k` in map `M`, or raise `KeyError` if not found.   | `__getitem__` method  |
| `M[k] = v`     | Associate value `v` with key `k` in map `M`, replacing it if key already exists.             | `__setitem__` method  |
| `del M[k]`     | Remove item with key `k` from map `M`, or raise `KeyError` if not found.                     | `__delitem__` method  |


 | Method        | Description                                                                                                                                                         |
|---------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `first()`     | Return the position containing the **least key**, or `None` if the tree is empty.                                                                                   |
| `last()`      | Return the position containing the **greatest key**, or `None` if the tree is empty.                                                                                |
| `before(p)`   | Return the position containing the **greatest key less than** the key at position `p` (the one visited immediately **before** `p` in an inorder traversal), or `None` if `p` is first. |
| `after(p)`    | Return the position containing the **least key greater than** the key at position `p` (the one visited immediately **after** `p` in an inorder traversal), or `None` if `p` is last. |


In [ ]:
Code Fragment11.1: Computing the successor of a position in a binary search tree.

In [ ]:
Algorithm after(p):
    if right(p) is not None then:
        # Successor is the leftmost node in p's right subtree
        walk = right(p)
        while left(walk) is not None do:
            walk = left(walk)
        return walk
    else:
        # Successor is the nearest ancestor for which p is in the left subtree
        walk = p
        ancestor = parent(walk)
        while ancestor is not None and walk == right(ancestor) do:
            walk = ancestor
            ancestor = parent(walk)
        return ancestor


In [ ]:
 Code Fragment 11.2: Recursive search in a binary search tree.

In [ ]:
Algorithm TreeSearch(T, p, k):
    if k == p.key():
        return p                        # successful search
    elif k < p.key() and T.left(p) is not None:
        return TreeSearch(T, T.left(p), k)    # search in left subtree
    elif k > p.key() and T.right(p) is not None:
        return TreeSearch(T, T.right(p), k)   # search in right subtree
    return p                            # unsuccessful search (hit a leaf)


In [ ]:
Code Fragment 11.3: Algorithm for inserting a key-value pair into a map that is
 represented as a binary search tree.

In [ ]:
Algorithm TreeInsert(T, k, v):
    Input: A search key k to be associated with value v
    
    p = TreeSearch(T, T.root(), k)
    
    if k == p.key():
        Set p’s value to v                # Key exists, update value
    elif k < p.key():
        Add node with item (k, v) as left child of p
    else:
        Add node with item (k, v) as right child of p


In [ ]:
Code Fragment 11.4: Beginning ofTreeMapclass including redefinedPosition
 class and nonpublic search utilities.
 

In [ ]:
# ----------------------------- TreeMap Definition -----------------------------
class TreeMap(LinkedBinaryTree, MapBase):
    """Sorted map implementation using a binary search tree."""

    # ---------------------- override Position class ----------------------
    class Position(LinkedBinaryTree.Position):
        def key(self):
            """Return key of map's key-value pair."""
            return self.element().key

        def value(self):
            """Return value of map's key-value pair."""
            return self.element().value

    # --------------------- nonpublic utilities -------------------------
    def subtree_search(self, p, k):
        """Return Position of p's subtree having key k, or last node searched."""
        if k == p.key():  # found match
            return p
        elif k < p.key():  # search left subtree
            if self.left(p) is not None:
                return self.subtree_search(self.left(p), k)
        else:  # search right subtree
            if self.right(p) is not None:
                return self.subtree_search(self.right(p), k)
        return p  # unsuccessful search

    def subtree_first_position(self, p):
        """Return Position of first item in subtree rooted at p."""
        walk = p
        while self.left(walk) is not None:  # keep walking left
            walk = self.left(walk)
        return walk

    def subtree_last_position(self, p):
        """Return Position of last item in subtree rooted at p."""
        walk = p
        while self.right(walk) is not None:  # keep walking right
            walk = self.right(walk)
        return walk


In [ ]:
Code Fragment 11.5: Positional methods first(), last(), before(p), after(p),
 and find position(p) accessor.


In [ ]:
    def first(self):
        """Return the first Position in the tree (or None if empty)."""
        return self.subtree_first_position(self.root()) if len(self) > 0 else None

    def last(self):
        """Return the last Position in the tree (or None if empty)."""
        return self.subtree_last_position(self.root()) if len(self) > 0 else None

    def before(self, p):
        """Return the Position just before p in the natural order."""
        self.validate(p)
        if self.left(p):
            return self.subtree_last_position(self.left(p))
        else:
            # walk upward
            walk = p
            above = self.parent(walk)
            while above is not None and walk == self.left(above):
                walk = above
                above = self.parent(walk)
            return above

    def after(self, p):
        """Return the Position just after p in the natural order."""
        # symmetric to before(p)
        if self.right(p):
            return self.subtree_first_position(self.right(p))
        else:
            walk = p
            above = self.parent(walk)
            while above is not None and walk == self.right(above):
                walk = above
                above = self.parent(walk)
            return above

    def find_position(self, k):
        """Return position with key k, or else neighbor (or None if empty)."""
        if self.isempty():
            return None
        else:
            p = self.subtree_search(self.root(), k)
            self.rebalance_access(p)  # hook for balanced tree subclasses
            return p


In [ ]:
 Code Fragment 11.6: Selected methods of the sorted map ADT: find min(),
 f
 ind ge(k),andfind range(start, stop); related methods
 are omitted for the sake of brevity.


In [ ]:
    def find_min(self):
        """Return (key, value) pair with minimum key (or None if empty)."""
        if self.isempty():
            return None
        else:
            p = self.first()
            return (p.key(), p.value())

    def find_ge(self, k):
        """Return (key, value) pair with least key >= k (or None if no such key)."""
        if self.isempty():
            return None
        else:
            p = self.find_position(k)
            if p.key() < k:
                p = self.after(p)
            return (p.key(), p.value()) if p is not None else None

    def find_range(self, start, stop):
        """Iterate all (key, value) pairs such that start <= key < stop."""
        if not self.isempty():
            if start is None:
                p = self.first()
            else:
                p = self.find_position(start)
                if p.key() < start:
                    p = self.after(p)
            while p is not None and (stop is None or p.key() < stop):
                yield (p.key(), p.value())
                p = self.after(p)


In [ ]:
 Code Fragment 11.7:
 getitem (k), setitem (k, v),and iter ().


In [ ]:
    def __getitem__(self, k):
        """Return value associated with key k (raise KeyError if not found)."""
        if self.isempty():
            raise KeyError('KeyError: ' + repr(k))
        else:
            p = self.subtree_search(self.root(), k)
            self.rebalance_access(p)
            if k != p.key():
                raise KeyError('KeyError: ' + repr(k))
            return p.value()

    def __setitem__(self, k, v):
        """Assign value v to key k, overwriting existing value if present."""
        if self.isempty():
            leaf = self.add_root(self.Item(k, v))
        else:
            p = self.subtree_search(self.root(), k)
            if p.key() == k:
                p.element().value = v
                self.rebalance_access(p)
                return
            else:
                item = self.Item(k, v)
                if p.key() < k:
                    leaf = self.add_right(p, item)
                else:
                    leaf = self.add_left(p, item)
                self.rebalance_insert(leaf)

    def __iter__(self):
        """Generate an iteration of all keys in the map in order."""
        p = self.first()
        while p is not None:
            yield p.key()
            p = self.after(p)


In [ ]:
 Code Fragment 11.8: Deletion either by position, as delete(p), orbykey, as
 delitem (k)

In [ ]:
    def delete(self, p):
        """Remove the item at given Position."""
        self.validate(p)
        if self.left(p) and self.right(p):  # has two children
            replacement = self.subtree_last_position(self.left(p))
            self.replace(p, replacement.element())
            p = replacement  # move to leaf to be deleted
        parent = self.parent(p)
        self.delete(p)  # inherited delete (now has ≤ 1 child)
        self.rebalance_delete(parent)

    def __delitem__(self, k):
        """Remove item associated with key k (raise KeyError if not found)."""
        if not self.isempty():
            p = self.subtree_search(self.root(), k)
            if k == p.key():
                self.delete(p)
                return
            self.rebalance_access(p)
        raise KeyError('KeyError: ' + repr(k))


| Operation                                       | Running Time |
|------------------------------------------------|---------------|
| `k in T`                                       | O(h)          |
| `T[k]`, `T[k] = v`                              | O(h)          |
| `T.delete(p)`, `del T[k]`                      | O(h)          |
| `T.find_position(k)`                           | O(h)          |
| `T.first()`, `T.last()`, `T.find_min()`, `T.find_max()` | O(h) |
| `T.before(p)`, `T.after(p)`                    | O(h)          |
| `T.find_lt(k)`, `T.find_le(k)`, `T.find_gt(k)`, `T.find_ge(k)` | O(h) |
| `T.find_range(start, stop)`                    | O(s + h)      |
| `iter(T)`, `reversed(T)`                       | O(n)          |


In [ ]:
 Table 11.1: Worst-case running times of the operations for aTreeMap T. We denote
 the current height of the tree with h, and the number of items reported byfind range
 as s. The space usage is O(n),wheren is the number of items stored in the map

In [ ]:
 Code Fragment 11.9: The trinode restructuring operation in a binary search tree.

In [ ]:
### Algorithm: restructure(x)

**Input:**  
A position `x` of a binary search tree `T` that has both a parent `y` and a grandparent `z`.

**Output:**  
Tree `T` after a trinode restructuring (single or double rotation) involving positions `x`, `y`, and `z`.

---

#### Steps:

1. Let `(a, b, c)` be a left-to-right **(inorder)** listing of the positions `x`, `y`, and `z`,  
   and let `(T1, T2, T3, T4)` be a left-to-right **(inorder)** listing of the four subtrees of `x`, `y`, and `z` not rooted at `x`, `y`, or `z`.

2. Replace the subtree rooted at `z` with a new subtree rooted at `b`.

3. Let `a` be the **left child** of `b`, and let `T1` and `T2` be the **left and right subtrees** of `a`, respectively.

4. Let `c` be the **right child** of `b`, and let `T3` and `T4` be the **left and right subtrees** of `c`, respectively.


In [ ]:
Code Fragment 11.10: Rebalancing Hook Stubs in TreeMap

In [ ]:
class TreeMap(LinkedBinaryTree, MapBase):
    """Sorted map implementation using a binary search tree."""

    # --- (Other methods from previous fragments would be here) ---

    def rebalance_insert(self, p):
        """Hook for rebalancing after an insertion.
        
        To be overridden in subclasses like AVLTreeMap or RedBlackTreeMap.
        """
        pass

    def rebalance_delete(self, p):
        """Hook for rebalancing after a deletion.
        
        To be overridden in subclasses for maintaining balance.
        """
        pass

    def rebalance_access(self, p):
        """Hook for rebalancing after an access.
        
        Used in some trees (e.g., splay trees) where accessing a node can trigger reorganization.
        """
        pass


In [ ]:
Code Fragment 11.11: Rotation and Restructuring in TreeMap

In [ ]:
class TreeMap(LinkedBinaryTree, MapBase):
    """Sorted map implementation using a binary search tree."""

    # --- (Earlier methods defined in previous code fragments) ---

    def relink(self, parent, child, make_left_child):
        """Relink parent node with child node (we allow child to be None)."""
        if make_left_child:                # make it a left child
            parent.left = child
        else:                              # make it a right child
            parent.right = child
        if child is not None:              # make child point to parent
            child.parent = parent

    def rotate(self, p):
        """Rotate Position p above its parent."""
        x = p.node
        y = x.parent                      # we assume this exists
        z = y.parent                      # grandparent (possibly None)

        if z is None:
            self.root = x                # x becomes root
            x.parent = None
        else:
            self.relink(z, x, y == z.left)  # x becomes direct child of z

        # Now rotate x and y, including transfer of middle subtree
        if x == y.left:
            self.relink(y, x.right, True)   # x.right becomes left child of y
            self.relink(x, y, False)        # y becomes right child of x
        else:
            self.relink(y, x.left, False)   # x.left becomes right child of y
            self.relink(x, y, True)         # y becomes left child of x

    def restructure(self, x):
        """Perform trinode restructuring of Position x with parent/grandparent."""
        y = self.parent(x)
        z = self.parent(y)

        if (x == self.right(y)) == (y == self.right(z)):
            # Matching alignments: single rotation (of y)
            self.rotate(y)
            return y                        # y is new subtree root
        else:
            # Opposite alignments: double rotation (of x)
            self.rotate(x)
            self.rotate(x)
            return x                        # x is new subtree root


In [ ]:
Table 11.2: Worst-case running times of operations for an n-item sorted map real
ized as an AVLtree T, with s denoting the number of items reported by find range.

| Operation                                | Running Time     |
|------------------------------------------|------------------|
| `k in T`                                 | O(log n)         |
| `T[k] = v`                               | O(log n)         |
| `T.delete(p)`, `del T[k]`                | O(log n)         |
| `T.find_position(k)`                     | O(log n)         |
| `T.first()`, `T.last()`                  | O(log n)         |
| `T.find_min()`, `T.find_max()`           | O(log n)         |
| `T.before(p)`, `T.after(p)`              | O(log n)         |
| `T.find_lt(k)`, `T.find_le(k)`           | O(log n)         |
| `T.find_gt(k)`, `T.find_ge(k)`           | O(log n)         |
| `T.find_range(start, stop)`             | O(s + log n)     |
| `iter(T)`, `reversed(T)`                 | O(n)             |


In [ ]:
 CodeFragment11.12 -11.13 :AVLTreeMapclass

In [ ]:
class AVLTreeMap(TreeMap):
    """Sorted map implementation using an AVL tree."""

    # -------------------------- nested Node class --------------------------
    class Node(TreeMap.Node):
        """Node class for AVL maintains height value for balancing."""
        __slots__ = '_height'  # additional data member to store height

        def __init__(self, element, parent=None, left=None, right=None):
            super().__init__(element, parent, left, right)
            self.height = 0  # will be recomputed during balancing

        def left_height(self):
            return self.left.height if self.left is not None else 0

        def right_height(self):
            return self.right.height if self.right is not None else 0

    # ---------------------- positional-based utility methods ----------------------
    def recompute_height(self, p):
        p.node.height = 1 + max(p.node.left_height(), p.node.right_height())

    def isbalanced(self, p):
        return abs(p.node.left_height() - p.node.right_height()) <= 1

    def tall_child(self, p, favorleft=False):
        """Return taller child; if equal, favor left if specified."""
        if p.node.left_height() + (1 if favorleft else 0) > p.node.right_height():
            return self.left(p)
        else:
            return self.right(p)

    def tall_grandchild(self, p):
        child = self.tall_child(p)
        alignment = (child == self.left(p))  # True if child is left child
        return self.tall_child(child, alignment)

    # ---------------------------- rebalancing logic ----------------------------
    def rebalance(self, p):
        while p is not None:
            old_height = p.node.height
            if not self.isbalanced(p):
                # imbalance detected — perform trinode restructuring
                p = self.restructure(self.tall_grandchild(p))
                # recompute heights after restructuring
                self.recompute_height(self.left(p))
                self.recompute_height(self.right(p))
            self.recompute_height(p)

            # if height hasn't changed, we can stop
            if p.node.height == old_height:
                p = None
            else:
                p = self.parent(p)

    # ---------------------- override balancing hooks ----------------------
    def rebalance_insert(self, p):
        self.rebalance(p)

    def rebalance_delete(self, p):
        self.rebalance(p)


In [ ]:
Code Fragment 11.14 -- SplayTreeMap — Self-Adjusting Binary Search Tree

In [ ]:
class SplayTreeMap(TreeMap):
    """Sorted map implementation using a splay tree."""

    # ----------------------------- Splay Operation -----------------------------
    def splay(self, p):
        while p != self.root():
            parent = self.parent(p)
            grand = self.parent(parent)

            if grand is None:
                # Zig case — single rotation (p and parent only)
                self.rotate(p)
            elif (parent == self.left(grand)) == (p == self.left(parent)):
                # Zig-Zig case — double rotation (same direction)
                self.rotate(parent)  # Move parent up
                self.rotate(p)       # Move p up
            else:
                # Zig-Zag case — double rotation (opposite direction)
                self.rotate(p)       # Move p up once
                self.rotate(p)       # Move p up again

    # --------------------- Override Balancing Hooks ---------------------
    def rebalance_insert(self, p):
        self.splay(p)

    def rebalancedelete(self, p):
        if p is not None:
            self.splay(p)

    def rebalanceaccess(self, p):
        self.splay(p)


In [ ]:
Red-Black Trees: Formal Properties

A Red-Black Tree is a type of balanced binary search tree that ensures logarithmic height through a set of color-based invariants. Each node in the tree is colored either red or black, and the tree adheres to the following three key properties:

1. 🖤 Root Property

The root must always be black.

This provides a consistent base for calculating black-depth and maintaining balance.

2. 🔴 Red Property

The children of a red node must be black.

This prevents consecutive red nodes (i.e., red-red violations), ensuring the tree does not lean too heavily in one direction.

3. ⚖️ Depth Property (Black-Depth Invariant)

All paths from a node to its descendant leaves (null or external nodes) must contain the same number of black nodes.

This black depth is defined as the count of black ancestors from a node up to the root (inclusive).

In [ ]:
 CodeFragment11.15:Beginning of the Red Black Tree Map class.
11.16 continuation of the Red Black Tree Map Class
11.7 Conclusion of the Red Black Tree Map Class

In [ ]:
# Red-Black TreeMap Implementation

class RedBlackTreeMap(TreeMap):
    """Sorted map implementation using a red-black tree."""

    class Node(TreeMap.Node):
        """Node class for red-black tree maintains bit that denotes color."""
        __slots__ = '_red'  # additional data member to store color bit

        def __init__(self, element, parent=None, left=None, right=None):
            super().__init__(element, parent, left, right)
            self._red = True  # new node is red by default

    # ------------------------- Positional-based utility methods ------------------------

    def _set_red(self, p): p._node._red = True

    def _set_black(self, p): p._node._red = False

    def _set_color(self, p, make_red): p._node._red = make_red

    def _is_red(self, p): return p is not None and p._node._red

    def _is_red_leaf(self, p): return self._is_red(p) and self.is_leaf(p)

    def _get_red_child(self, p):
        """Return a red child of p (or None if no such child)."""
        for child in (self.left(p), self.right(p)):
            if self._is_red(child):
                return child
        return None

    # ------------------------- Support for insertions ------------------------

    def _rebalance_insert(self, p):
        self._resolve_red(p)  # new node is always red

    def _resolve_red(self, p):
        if self.is_root(p):
            self._set_black(p)  # make root black
        else:
            parent = self.parent(p)
            if self._is_red(parent):  # double red problem
                uncle = self.sibling(parent)
                if not self._is_red(uncle):  # Case 1: misshapen 4-node
                    middle = self._restructure(p)  # do trinode restructuring
                    self._set_black(middle)  # fix colors
                    self._set_red(self.left(middle))
                    self._set_red(self.right(middle))
                else:  # Case 2: overfull 5-node
                    grand = self.parent(parent)
                    self._set_red(grand)
                    self._set_black(self.left(grand))
                    self._set_black(self.right(grand))
                    self._resolve_red(grand)  # recur at red grandparent

    # ------------------------- Support for deletions ------------------------

    def _rebalance_delete(self, p):
        if len(self) == 1:
            self._set_black(self.root())  # ensure root is black in special case
        elif p is not None:
            n = self.num_children(p)
            if n == 1:  # deficit exists unless child is a red leaf
                c = next(self.children(p))
                if not self._is_red_leaf(c):
                    self._fix_deficit(p, c)
            elif n == 2:  # removed black node with red child
                if self._is_red_leaf(self.left(p)):
                    self._set_black(self.left(p))
                else:
                    self._set_black(self


In [ ]:
### Exercises R.11.1-11.28

In [ ]:
R-11.1 If we insert the entries (1,A), (2,B), (3,C), (4,D),and(5,E),in this order,
 into an initially empty binary search tree, what will it look like?

In [ ]:
(1, A)
    \
   (2, B)
        \
       (3, C)
            \
           (4, D)
                \
               (5, E)


In [ ]:
R-11.2 Insert, into an empty binary search tree, entries with keys 30, 40, 24, 58,
 48, 26, 11, 13 (in this order). Draw the tree after each insertion.


In [ ]:
30

30
  \
   40

   30
  /  \
24    40

     30
    /  \
  24    40
           \
            58

     30
    /  \
  24    40
           \
            58
           /
         48

     30
    /  \
  24    40
    \     \
    26     58
          /
        48

     30
    /  \
  24    40
 /  \     \
11  26     58
          /
        48


     30
    /  \
  24    40
 /  \     \
11  26     58
  \       /
  13     48


         30
       /    \
     24      40
    /  \        \
  11   26       58
    \          /
    13       48



In [ ]:
 R-11.3 How many different binary search trees can store the keys {1,2,3}?

In [ ]:
the number of the difference binary search trees can be formed using the catalan number Cn

Cn = 1/(n+1)(2n n)

{1,2,3} , n=3

C3 = 1/4 (6 3) = 1/4 *20 =5

you get 20, since (6 3) is the binomail coefficient "i did not put it the correct way here, as I don't know how"

6!=720
3! =6
= 6!/(3!(6-3)) = 6!/3!3!
720/6*6 = 20

So, you can form 5 different binary search trees wiht the stored keys 1,2,3

In [ ]:
R-11.4 Dr.Amongus claims that the order in which a fixed set of entries is inserted
 into a binary search tree does not matter—the same tree results every time.
 Give a small example that proves he is wrong.


In [5]:
Dr. Amongus is wrong, 

Using this set as an example 1.2.3
Insertion order matters. Each permutation gives a different tree structure. 

    1
     \
      2
       \
        3
    2
   / \
  1   3

    3
   /
  2
 /
1

Based on the example we did before using catanlan formula, you can see there are multiple ways to create the trees.
Using that as example, as well, we get 5 different trees

NameError: name 'Dr' is not defined

In [ ]:
 R-11.5 Dr.Amongus claims that the order in which a fixed set of entries is inserted
 into an AVL tree does not matter—the same AVL tree results every time.
 Give a small example that proves he is wrong.


In [ ]:
Dr.Amongus is very good at being wrong, for AVL trees, they are similar to binary trees and that the order
truly depends on the insertion of the trees,using 1,2,3 as an example again.

AVL tree structure does depend on the order of insertions, because rotations are triggered differently depending on when and where imbalances occur.
insert 1
1

insert 2
  1
   \
    2
insert 3

  2
 / \
1   3

Now we will do it again this time 3,2,1

  2
 / \
1   3

this time 2,1,3

  2
 / \
1   3
So yes, this still ends in the same balanced tree, but the rotations and structure during insertion are differetn

In [ ]:
 R-11.6 Our implementation of the TreeMap. subtree search utility, from Code
 Fragment 11.4, relies on recursion. For a large unbalanced tree, Python’s
 default limit on recursive depth may be prohibitive. Give an alternative
 implementation of that method that does not rely on the use of recursion.
 

In [ ]:
def subtree_search(tree, p, k):
    """Return Position of the item with greatest key <= k in subtree rooted at p."""
    walk = p
    candidate = None

    while walk is not None:
        if k < walk.key():
            if tree.left(walk) is not None:
                walk = tree.left(walk)
            else:
                break
        else:
            candidate = walk
            if tree.right(walk) is not None:
                walk = tree.right(walk)
            else:
                break

    return candidate


In [ ]:
R-11.7 Do the trinode restructurings in Figures 11.12 and 11.14 result in single
 or double rotations?


In [ ]:
Figure 11.12 results in a single rotation.

Figure 11.14 results in a double rotation.

In [ ]:
 R-11.8 Draw the AVL tree resulting from the insertion of an entry with key 52
 into the AVL tree of Figure 11.14b.


In [ ]:
         62
        /  \
      44    78
     /  \     \
   17   52     88
        /  \
      50   54
     /
   48


In [ ]:
 R-11.9 Draw the AVL tree resulting from the removal of the entry with key 62
 from the AVL tree of Figure 11.14b.
 

In [ ]:
         54
       /    \
     50      78
    /  \        \
  44   —       88
 /  \
17  48


In [ ]:
R-11.10 Explain why performing a rotation in an n-node binary tree when using
 the array-based representation of Section 8.3.2 takes Ω(n) time.
 

In [ ]:
Rotations in array-based binary trees take Ω(n) time because the structure relies on positional indices to define relationships, and rebalancing operations (like rotations) can require shifting many elements to maintain those index rules.

In [ ]:
R-11.11 Give a schematic figure, in the style of Figure 11.13, showing the heights
 of subtrees during a deletion operation in an AVL tree that triggers a tri
node restructuring for the case in which the two children of the node de
noted as y start with equal heights. What is the net effect of the height of
 the rebalanced subtree due to the deletion operation?


In [ ]:
The height of the rebalanced subtree decreases by 1, from h+2 to h+1.

In [ ]:
 R-11.12 Repeat the previous problem, considering the case in which y’s children
 start with different heights.


In [ ]:
Before: Height of subtree rooted at z = h+2

After: Height of subtree rooted at y = h+1

In [ ]:

 R-11.13 The rules for a deletion in an AVL tree specifically require that when the
 two subtrees of the node denoted as y have equal height, child x should be
 chosen to be “aligned” with y (so that x and y are both left children or both
 right children). To better understand this requirement, repeat Exercise R
11.11 assuming we picked the misaligned choice of x. Why might there
 be a problem in restoring the AVL property with that choice?


In [ ]:
If the children of node y have equal height and we pick the misaligned child x during a deletion operation in an AVL tree, we may fail to restore the AVL property.

In [ ]:
 R-11.14 Perform the following sequence of operations in an initially empty splay
 tree and draw the tree after each set of operations.
 a. Insert keys 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, in this order.
 b. Search for keys 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, in this order.
 c. Delete keys 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, in this order.


| Step | Description                  | Tree Shape                        |
| ---- | ---------------------------- | --------------------------------- |
| A    | Insert 0 to 18               | Left-leaning linked list          |
| B    | Search 1–19 (odds, all fail) | Frequent rotations; tree balances |
| C    | Delete 0 to 18               | Tree becomes empty                |


In [ ]:
 R-11.15 What does a splay tree look like if its entries are accessed in increasing
 order by their keys?


In [ ]:
Access 1:
1

Access 2 (splay 2 to root):
   2
  /
1

Access 3 (splay 3 to root):
     3
    /
   2
  /
1

Access 4 (splay 4 to root):
       4
      /
     3
    /
   2
  /
1

Access 5 (splay 5 to root):
         5
        /
       4
      /
     3
    /
   2
  /
1


In [ ]:
 R-11.16 Is the search tree of Figure 11.23(a) a (2,4) tree? Why or why not?
 

In [8]:
No, the search tree in Figure 11.23(a) is not a (2,4) tree.

this is because a a 1-4 tree must follow certain rules,
every node has to between 2 and 4 children
every internal node has to sotre one fewer key than its number of children,
all external nodes aka leaves, are at the same depth


Not all internal nodes have the correct number of children for their number of keys.

One or more internal nodes are malformed under the (2,4)-tree rules, thius is why it is not a valid 2-4 tree


SyntaxError: invalid syntax (3819895100.py, line 1)

In [10]:
R-11.17 An alternative way of performing a split at a node w in a (2,4) tree is
 to partition w into w and w , with w being a 2-node and w a 3-node.
 Which of the keys k1, k2, k3,ork4 do we store at w’s parent? Why?


SyntaxError: invalid syntax (1072445862.py, line 1)

In [ ]:
We store k₂ at w’s parent.

In [ ]:
 R-11.18 Dr. Amongus claims that a (2,4) tree storing a set of entries will always
 have the same structure, regardless of the order in which the entries are
 inserted. Show that he is wrong.


In [12]:
The order of insertions does affect the structure of a (2,4) tree. While the tree always remains balanced and satisfies its properties, different insertion orders can lead to different structures due to when and where node splits occur.

Insert 1 → root = [1]

Insert 2 → root = [1, 2]

Insert 3 → root = [1, 2, 3]

Insert 4 → root splits → promote 2
      [2]
     /   \
  [1]   [3, 4]


Insert 2 → root = [2]

Insert 1 → root = [1, 2] (after reordering)

Insert 4 → root = [1, 2, 4]

Insert 3 → now root becomes [1, 2, 3, 4] → must split

      [2]
     /   \
  [1]   [3, 4]


SyntaxError: invalid syntax (233193465.py, line 1)

In [14]:
 R-11.19 Draw four different red-black trees that correspond to the same (2,4) tree.


SyntaxError: invalid syntax (2830347055.py, line 1)

In [ ]:
      20(B)
     /    \
  10(R)  30(R)


      20(B)
     /    \
  30(R)  10(R)

      10(B)
         \
         20(R)
            \
            30(R)

        30(B)
       /
    20(R)
   /
10(R)


In [ ]:
 R-11.20 Consider the set of keys K = {1,2,3,4,5,6,7,8,9,10,11,12,13,14,15}.
 a. Draw a (2,4) tree storing K as its keys using the fewest number of
 nodes.
 b. Draw a (2,4) tree storing K as its keys using the maximum number
 of nodes.


In [ ]:
                      [ 8 ]
              /         |         \
        [2,4,6]     [10,12,14]   [ ]
       /   |   \      /   |   \     
     [1] [3,5] [7]  [9] [11,13] [15]





[8]
               /     \
            [4]       [12]
           /   \     /    \
        [2]   [6] [10]   [14]
       / \   / \   / \    / \
     [1][3][5][7][9][11][13][15]


In [ ]:
 R-11.21 Consider the sequence of keys (5,16,22,45,2,10,18,30,50,12,1).Draw
 the result of inserting entries with these keys (in the given order) into
 a. An initially empty (2,4) tree.
 b. An initially empty red-black tree.
 

In [ ]:
                 [5, 16, 22]
              /     |     |     \
        [1,2]  [10,12]  [18]  [30,45,50]


              [16B]
           /         \
       [5B]         [22B]
      /   \        /     \
   [2R] [10R]  [18R] [45B]
   /         \             /   \
[1B]      [12B]     [30R] [50R]


In [ ]:
R-11.22 For the following statements about red-black trees, provide a justification
 for each true statement and a counterexample for each false one.
 a. Asubtree of a red-black tree is itself a red-black tree.
 b. Anode that does not have a sibling is red.
 c. There is a unique (2,4) tree associated with a given red-black tree.
 d. There is a unique red-black tree associated with a given (2,4) tree.


In [ ]:
a. false 
 root is black, no two red nodes appear consecutively, and every path from node to leagf has the same number of black nodes

b. false
sibling existence has nothing to do with color

c. True
each black node becomes a node in a 2-4 tree, and red children are goruped with their black parent

d. false
Many red black trees can represent the same 2-4 nmode

In [ ]:
 R-11.23 Explain why you would get the same output in an inorder listing of the
 entries in a binary search tree, T, independent of whether T is maintained
 to be an AVL tree, splay tree, or red-black tree.

 

In [ ]:
Regardless of whether a binary search tree (BST) is implemented as an AVL tree, a splay tree, or a red-black tree, the inorder traversal will always produce the same output as long as the keys inserted are the same.



In [ ]:
R-11.24 Consider a tree T storing 100,000 entries. What is the worst-case height
 of T in the following cases?
 a. T is a binary search tree.
 b. T is an AVLtree.
 c. T is a splay tree.
 d. T is a (2,4) tree.
 e. T is a red-black tree.


In [ ]:
a. h=n=100,000
b. 24
c. 100,000
d.17
e.34


In [ ]:
 R-11.25 Draw an example of a red-black tree that is not an AVL tree.


In [ ]:
        10(B)
       /    \
    5(R)    15(B)
   /         
 2(B)       
/
1(R)


In [ ]:
 R-11.26 Let T be a red-black tree and let p be the position of the parent of the
 original node that is deleted by the standard search tree deletion algorithm.
 Prove that if p has zero children, the removed node was a red leaf.


In [ ]:
red leaves can be removed without violating the red black tree properties, the removal of a red leaf doesn affect the balck node height, so there is no rebalancing needed, so after the deletion p would have zero childrten adn there would bne no vioation